In [ ]:
import numpy as np
import importlib
import dataset
import config, plotting, sample, SampleCharacter, XLim, QRselection
import QrModels, Point
import copy
import RelativeImportance

importlib.reload(config)
importlib.reload(dataset)
importlib.reload(plotting)
importlib.reload(sample)
importlib.reload(SampleCharacter)
importlib.reload(XLim)
importlib.reload(QRselection)
importlib.reload(QrModels)
importlib.reload(Point)
importlib.reload(RelativeImportance)

In [ ]:
import os
dataset_config = config.DatasetConfig(scenario="PG", n_instance=1000)

assert(dataset_config.scenario == "PG" 
       or dataset_config.scenario == "PT"
      or dataset_config.scenario == "MLCC"
      )
fig_dir = f"../figures/{dataset_config.scenario}"

try:
    os.mkdir(fig_dir)
    print(f"Directory {fig_dir} created ") 
except FileExistsError:
    print(f"Directory {fig_dir} already exists replacing files in this notebook")


In [ ]:
file_name_test = "../data/functional_3d/test_data.txt"
X_test,Y_test = dataset.get_functional_test_data(file_name_test)

In [ ]:
file_name_train = "../data/functional_3d/train_data.txt"
X_train,Y_train = dataset.get_functional_train_data(file_name_train)

In [ ]:
random_seed = 1985
if dataset_config.scenario == "PG" :
    exp_config = config.Config(
        model=config.ModelConfig(activation="elu", lr_gen=0.0001, lr_disc=0.001, dec_gen=0, dec_disc=0, 
                                 optim_gen="Adam", optim_disc="Adam", z_input_size=1, random_seed=random_seed),
        training=config.TrainingConfig(n_epochs=500, batch_size=100, n_samples=50),
        dataset=dataset_config,
        run=config.RunConfig(save_fig=1)
    )
    
elif dataset_config.scenario == "PT":
    exp_config = config.Config(
        model=config.ModelConfig(activation="elu", lr_gen=0.0001, lr_disc=0.0005, dec_gen=0, dec_disc=0, 
                                 optim_gen="Adam", optim_disc="Adam", z_input_size=1, random_seed=random_seed),
        training=config.TrainingConfig(n_epochs=500, batch_size=100, n_samples=50),
        dataset=dataset_config,
        run=config.RunConfig(save_fig=1)
    )
elif dataset_config.scenario == "MLCC":
    exp_config = config.Config(
        model=config.ModelConfig(activation="elu", lr_gen=0.0001, lr_disc=0.0005, dec_gen=0, dec_disc=0, 
                                 optim_gen="Adam", optim_disc="Adam", z_input_size=1, random_seed=random_seed),
        training=config.TrainingConfig(n_epochs=500, batch_size=100, n_samples=50),
        dataset=dataset_config,
        run=config.RunConfig(save_fig=1)
    )  

In [ ]:
coef = sample.get_sta_reg_cov(X_train,Y_train)

In [ ]:
imp = RelativeImportance.relativeImp_3d()
imp = list(imp.values())
print("The importance for every dimension:")
print(imp)

In [ ]:
length, max_dist = sample.get_sample_length(X_train,imp)

In [ ]:
x_min = np.amin(X_train, axis=0)
x_max = np.amax(X_train, axis=0)
L = sample.get_x_len(x_min, x_max)

In [ ]:
n_sample,length = sample.divide_sample(length, L)

In [ ]:
dim = len(X_train[0])
gen_x = sample.gen_x_center(dim,length,n_sample, x_min)

In [ ]:
gen_sample_point = sample.gen_product(gen_x)
#print("样方中心点：")
gen_sample_point = np.array(gen_sample_point)
#print(gen_sample_point)

In [ ]:
xlimit = []
for index in range(len(n_sample)):
    l = []
    for i in range(n_sample[index]):
        x = x_min[index] + i * length[index]
        l.append(x)
    l.append(x_min[index] + n_sample[index] * length[index])
    xlimit.append(l)

In [ ]:
sample_list = []
xlim_list = []

In [ ]:
x_value = []
for index in range(dim):
    xl = []
    for i in range(len(X_train)):
        x = X_train[i][index]
        r = 0
        for xi in xl:
            if xi == x:
                r = 1
        if r == 0:
            xl.append(x)
    x_value.append(xl)
    
x_value_ori = copy.deepcopy(x_value)

In [ ]:
XLim.con_s(gen_sample_point, sample_list, dim, xlimit)

In [ ]:
XLim.con_sample(xlim_list, length, x_min, dim, n_sample)

In [ ]:
XLim.sample_feature(xlim_list, sample_list, x_value)

In [ ]:
gen_x_cross = sample.gen_product(x_value)

In [ ]:
f_list = []
sample.cross_point_del(gen_x_cross, X_train)
sample.point_filiter(gen_x_cross, X_train, max_dist, x_value, x_value_ori, dim, f_list)

In [ ]:
plot_xlim = []
for index in range(len(xlimit)):
    pl = []
    for i in range(len(xlimit[index])):
        if i != len(xlimit[index]) - 1:
            pl.append(xlimit[index][i])
        else:
            pl.append(x_max[index])
    plot_xlim.append(pl)
        

In [ ]:
gen_x_cross = np.array(gen_x_cross)

In [ ]:
XLim.xl_attri(xlim_list, X_train, gen_x_cross)
XLim.sample_attri(sample_list, X_train, gen_x_cross, Y_train)
XLim.add_xvalue(xlim_list)

In [ ]:
import GPy

noise = 0.01
length_ = 0.1

run_hyperopt_search = True

kernel = GPy.kern.RBF(input_dim=3, variance=noise, lengthscale=length_)
gpr = GPy.models.GPRegression(X_train, Y_train.reshape(-1,1), kernel)
if run_hyperopt_search:
    gpr.optimize(messages=True) 

In [ ]:
gen_y_cross, cov_train_cross = gpr.predict(np.array(gen_x_cross))

In [ ]:
XLim.add_y(sample_list, gpr)
print("over")

In [ ]:
point_list = []
Point.con_point(gen_x_cross, gen_y_cross, point_list)
print("over")

In [ ]:
for sample in sample_list:
    for index in range(len(sample.gen_xlist)):
        for point in point_list:
            r = -1
            for i in range(dim):
                if point.x[i] != sample.gen_xlist[index][i]:
                    r = 0
                    break
            if r == -1:
                sample.points.append(point)
print("over")

In [ ]:
import statsmodels.api as sm
qrX = X_train
qrX = sm.add_constant(qrX[0:])
qr = sm.QuantReg(Y_train.reshape(-1,1),qrX)
res = qr.fit(q=.2)
print(res.summary())

In [ ]:
quantiles = np.arange(.05, .96, .1)
quantiles = np.around(quantiles, decimals=3)
def fit_model(q):
    res = qr.fit(q=q)
    return q, np.around(res.params,decimals=  4)

models = []

for x in quantiles:
    q, param = fit_model(x)
    model = QrModels.QrModels(q, param[0], param[1:])
    models.append(model)

for model in models:
    print(str(model.q)+'\t'+str(model.a)+'\t'+str(model.param))

ols = sm.OLS(Y_train.reshape(-1,1),qrX).fit()
           
for ol in ols.params:
    print(str(ol))
print("over")

In [ ]:
y_quantile = [-3.586, -1.143, -0.554, -0.179, 0.002, 0.027, 0.113, 0.307, 0.693, 2.900]

In [ ]:
vir_xpoint = []
vir_ypoint = []

for xv in x_value:
    xv.sort()
for xv in x_value_ori:
    xv.sort()
print("over")

In [ ]:
import QRselection
import importlib
importlib.reload(QRselection)


QRselection.qr_selection(xlim_list, models, vir_xpoint, vir_ypoint, y_quantile, ols, x_value_ori,
                 n_sample, X_train, Y_train, sample_list, point_list, x_value)
print("over")

In [ ]:
i = 0
for point in point_list:
    if point.checked == 1:
        i += 1
print("over")

In [ ]:
del_point = []
vir_point = []

for point in point_list:
    point.true = dataset.function_3d(point.x[0], point.x[1], point.x[2])
    point.erro = point.true - point.y
    if point.y not in vir_ypoint:
        del_point.append(point)
    else:
        vir_point.append(point)
print("over")

In [ ]:
plotting.plot_qr(del_point, exp_config, fig_dir, "qr")
plotting.plot_qr(vir_point, exp_config, fig_dir, "left")
plotting.plot_erro(vir_point, exp_config, fig_dir, "erro")

In [ ]:
exist_list = []
for sample in sample_list:
    points = []
    for index in range(len(sample.points)):
        if sample.points[index] in vir_point:
            points.append(sample.points[index])
    if len(points) > 0:
        if sample.ori_num == 0:
            i = np.random.randint(0, len(points))
            exist_list.append(points[i])
        else:
            #i = np.random.randint(0, len(points), size = sample.ori_num + 1)
            i = len(points)
            for j in range(i):
                exist_list.append(points[j])

e_list = []
for point in exist_list:
    e_list.append(point.x)      
print("over")

In [ ]:
def mae1(point_list):
    p = len(point_list)
    a = 0
    for point in point_list:
        a += abs(point.y - point.true)
    return a / p

def mse1(point_list):
    p = len(point_list)
    a = 0
    for point in point_list:
        a += pow(point.y - point.true, 2)
    return a / p

mae = mae1(vir_point)
mse = mse1(vir_point)

print(" all vir point")
print(mae)
print(mse)

In [ ]:
mae_s = mae1(exist_list)
mse_s = mse1(exist_list)

print(" selected vir point")
print(mae_s)
print(mse_s)


In [ ]:
r = 0
for index in range(len(exist_list)):
    point = exist_list[index]
    a = abs(point.true - point.y)
    if a > r:
        r = a
        print(index)
        


